`Last update at 2021-03-16`

# Background

## Rounds to test
```
Round 1: 60 MAGs; 39 MS/MS-BGC links (1 true link)
Round 2: 279 genomes/MAGs; 1 MS/MS-BGC links (0 true link)
Round 3: 279 genomes/MAGs and 589 metagenomes; 16 MS/MS-BGC links (8 true link)
```

## Outline

```
    1) Downloading (meta)genomes and metagenome-assembled genomes (MAGs) that contain a paired untargeted 
    metabolomics (LC-MS/MS and in the future GC-MS/MS)(metabolomic files also going to be downloaded):
    1.1) Downloading all JSON files from PoDP;
    1.2) Parsing JSON files and creating podp_merged_df;
    1.3) Obtaining uniform Genbank code from BioSample ID; (estimated runtime of 02:46:30)
    1.4) Adding new column with the downloaded NCBI IDs;
    1.5) Downloading and unzipping the NCBI genomes;
    1.6) Copying previously downloaded genomes;
    1.7) Downloading metagenomic reads;
    1.8) Downloading JGI (meta)genomes;
    1.9) Separating Genomes without antiSMASH;
    1.10) Running antiSMASH;
    1.11) Filtering links for (meta)genomes with antiSMASH and downloading LC-MS/MS.
```

In [1]:
import pandas as pd
import time
import glob
import numpy as np
import csv
import subprocess
import glob
import os
import re
import networkx
from networkx.algorithms.components.connected import connected_components
from collections import defaultdict
import matplotlib.pyplot as plt
from Bio import SeqIO
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
import pickle
import json

In [2]:
# !mkdir ./inputs
# !mkdir ./outputs/
# !mkdir ./temp_files/

# 1. Downloading genomes from the Paired omics Data Platform (PoDP)

## 1.1. Downloading all JSON files from PoDP

```
Go to PoDP (https://pairedomicsdata.bioinformatics.nl) and download the database JSON files, move files to 
input folder (path/to/NPOmix/inputs/databases/).
```

## 1.2. Parsing JSON files and creating podp_merged_df

In [3]:
def get_strain_dicts(filename):
    strain_gbk_dict = {}
    strain_biosample_dict = {}
    strain_db_dict = {}
    strain_to_shotgun = {}
    with open(filename) as f:
        data1 = json.load(f)
        if 'message' not in data1.keys():
            data1 = data1['genomes']
            json_df1 = pd.DataFrame(data1)
            for i,r in json_df1.iterrows():
                genome_line = dict(json_df1['genome_ID'].loc[i])
                if 'ENA_NCBI_accession' in genome_line.keys():
                    genomeID = genome_line['ENA_NCBI_accession']
                    db_id = 'ENA_NCBI_accession'
                if 'GenBank_accession' in genome_line.keys():
                    genomeID = genome_line['GenBank_accession']
                    db_id = 'GenBank_accession'
                if 'RefSeq_accession' in genome_line.keys():
                    genomeID = genome_line['RefSeq_accession']
                    db_id = 'RefSeq_accession'
                if 'JGI_Genome_ID' in genome_line.keys():
                    genomeID = genome_line['JGI_Genome_ID']
                    db_id = 'JGI_Genome_ID'
                if 'JGI_ID' in genome_line.keys():
                    genomeID = genome_line['JGI_ID']
                    db_id = 'JGI_ID'
                if 'JGI_IMG_genome_ID'in genome_line.keys():
                    genomeID = genome_line['JGI_IMG_genome_ID']
                    db_id = 'JGI_IMG_genome_ID'
                strain_to_shotgun[json_df1['genome_label'].loc[i]] = genome_line['genome_type']
                strain_gbk_dict[json_df1['genome_label'].loc[i]] = genomeID
                strain_db_dict[json_df1['genome_label'].loc[i]] = db_id
                if 'BioSample_accession' in json_df1.columns:
                    strain_biosample_dict[json_df1['genome_label'].loc[i]] = json_df1['BioSample_accession'].loc[i]
                else:
                    strain_biosample_dict[json_df1['genome_label'].loc[i]] = 'No BioSample'
    return strain_gbk_dict,strain_biosample_dict,strain_db_dict,strain_to_shotgun

def get_paired_df(filename,strain_gbk_dict,strain_biosample_dict,strain_db_dict,strain_to_shotgun):
    col1,col2,col3,col4,col5,col6 = [],[],[],[],[],[]
    with open(filename) as f:
        data2 = json.load(f)['genome_metabolome_links']
        json_df2 = pd.DataFrame(data2)
        for i,r in json_df2.iterrows():
            col1.append(strain_gbk_dict[json_df2['genome_label'].loc[i]])
            col2.append(strain_db_dict[json_df2['genome_label'].loc[i]])
            col3.append(strain_biosample_dict[json_df2['genome_label'].loc[i]])
            col4.append(json_df2['metabolomics_file'].loc[i])
            col5.append(json_df2['genome_label'].loc[i])
            col6.append(strain_to_shotgun[json_df2['genome_label'].loc[i]])
    frames = {'NCBI_ID':col1, 'Database':col2, 'BioSample':col3, 
              'LCMS_file':col4, 'Genome_label':col5, 'Method':col6}
    paired_df = pd.DataFrame(frames)
    return paired_df

In [5]:
table_list = glob.glob('./inputs/PoDP_datasets/*.json')

for filename in table_list:
    strain_gbk_dict,strain_biosample_dict,strain_db_dict,strain_to_shotgun = get_strain_dicts(filename)
    if strain_gbk_dict:
        if filename == table_list[0]:
            podp_merged_df = get_paired_df(filename,strain_gbk_dict,strain_biosample_dict,strain_db_dict,strain_to_shotgun)
        else:
            paired_df = get_paired_df(filename,strain_gbk_dict,strain_biosample_dict,strain_db_dict,strain_to_shotgun)
            frames = [podp_merged_df,paired_df]
            podp_merged_df = pd.concat(frames)
        
podp_merged_df = podp_merged_df.reset_index(drop=True)

podp_merged_df

,NCBI_ID,Database,BioSample,LCMS_file,Genome_label,Method
0,GCA_000240165.1,RefSeq_accession,SAMN02603879,ftp://massive.ucsd.edu/MSV000078850/results/S....,S. cattleya,genome
1,GCA_000240165.1,RefSeq_accession,SAMN02603879,ftp://massive.ucsd.edu/MSV000078850/results/S....,S. cattleya,genome
2,GCA_000240165.1,RefSeq_accession,SAMN02603879,ftp://massive.ucsd.edu/MSV000078850/results/S....,S. cattleya,genome
3,GCA_000240165.1,RefSeq_accession,SAMN02603879,ftp://massive.ucsd.edu/MSV000078850/results/S....,S. cattleya,genome
4,GCA_000240165.1,RefSeq_accession,SAMN02603879,ftp://massive.ucsd.edu/MSV000078850/results/S....,S. cattleya,genome
...,...,...,...,...,...,...
4891,GCA_000286575.1,GenBank_accession,SAMN00255227,ftp://massive.ucsd.edu/MSV000084945/ccms_peak/...,Burkholderia multivorans CF2,genome
4892,GCA_000959505.1,GenBank_accession,SAMN03144971,ftp://massive.ucsd.edu/MSV000084945/ccms_peak/...,Burkholderia dolosa AU0158,genome
4893,GCA_000959505.1,GenBank_accession,SAMN03144971,ftp://massive.ucsd.edu/MSV000084945/ccms_peak/...,Burkholderia dolosa AU0158,genome
4894,GCA_003568605.1,GenBank_accession,SAMN02866398,ftp://massive.ucsd.edu/MSV000084945/ccms_peak/...,Burkholderia thailandensis E264,genome


In [6]:
len(table_list) ### contain all online datasets

71

In [7]:
online_massive = ['MTBLS1606','MSV000086050','MSV000085589','MSV000085586','MSV000085376','MSV000085214','MSV000085210','MSV000085192','MSV000085180','MSV000085179','MSV000085159','MSV000085158','MSV000085141','MSV000085123','MSV000085085','MSV000085038','MSV000085032','MSV000085031','MSV000085027','MSV000085026','MSV000085023','MSV000085021','MSV000085018','MSV000085003','MSV000085000','MSV000084989','MSV000084954','MSV000084950','MSV000084950','MSV000084945','MSV000084884','MSV000084781','MSV000084771','MSV000084723','MSV000084674','MSV000084475','MSV000084475','MSV000084117','MSV000083835','MSV000083734','MSV000083648','MSV000083387','MSV000083302','MSV000083295','MSV000083268','MSV000083081','MSV000082988','MSV000082969','MSV000082891','MSV000082831','MSV000082285','MSV000082045','MSV000081832','MSV000081504','MSV000081318','MSV000081063','MSV000080427','MSV000080251','MSV000080179','MSV000079519','MSV000079284','MSV000079139','MSV000079015','MSV000078995','MSV000078891','MSV000078850','MSV000078847','MSV000078839','MSV000078836','MSV000078667','MSV000078556']

In [8]:
downloaded_massive = []

for item in podp_merged_df['LCMS_file']:
    if 'MSV' in item:
        if 'ftp://maftp' in item:
            downloaded_massive.append(item[31:43])
        else:
            downloaded_massive.append(item[23:35])
    if 'MTBLS' in item:
        downloaded_massive.append(item[35:])
    
downloaded_massive = list(np.unique(downloaded_massive))

for item in online_massive:
    if item not in downloaded_massive:
        print(item)
### we downloaded and processed all datasets (69 unique and 71 total MassIVE/Metabolights IDs)

In [9]:
len(np.unique(podp_merged_df['NCBI_ID'])),len(podp_merged_df)
### unique genomes and total number of links (the second matches the online record)

(2289, 4896)

In [10]:
len(podp_merged_df[podp_merged_df['BioSample'] == 'No BioSample'])

799

In [11]:
len(np.unique(podp_merged_df[podp_merged_df['BioSample'] != 'No BioSample']['BioSample']))

1677

In [12]:
799+1677

2476

In [13]:
for item in np.unique(podp_merged_df['Database']):
    print(item, len(podp_merged_df[podp_merged_df['Database'] == item]['NCBI_ID']))

ENA_NCBI_accession 1298
GenBank_accession 2107
JGI_Genome_ID 765
JGI_ID 2
JGI_IMG_genome_ID 612
RefSeq_accession 112


In [14]:
for item in np.unique(podp_merged_df['Method']):
    print(item, len(podp_merged_df[podp_merged_df['Method'] == item]['NCBI_ID']))

genome 3456
metagenome 1306
metagenome-assembled genome 134


## 1.3. Obtaining uniform Genbank code from BioSample ID

```
Separating JGI/ENA IDs from NCBI genomes, these will be downloaded in another step, for now we'll focus on NCBI genomes
```

In [15]:
filtered_df = podp_merged_df[podp_merged_df['Database'] != 'JGI_IMG_genome_ID']
filtered_df = filtered_df[filtered_df['Database'] != 'JGI_Genome_ID']
filtered_df = filtered_df[filtered_df['Database'] != 'JGI_ID']
filtered_df = filtered_df[filtered_df['Database'] != 'ENA_NCBI_accession']
filtered_df = filtered_df[filtered_df['BioSample'] != 'No BioSample']
filtered_df = filtered_df.reset_index(drop=True)

filtered_df

,NCBI_ID,Database,BioSample,LCMS_file,Genome_label,Method
0,GCA_000240165.1,RefSeq_accession,SAMN02603879,ftp://massive.ucsd.edu/MSV000078850/results/S....,S. cattleya,genome
1,GCA_000240165.1,RefSeq_accession,SAMN02603879,ftp://massive.ucsd.edu/MSV000078850/results/S....,S. cattleya,genome
2,GCA_000240165.1,RefSeq_accession,SAMN02603879,ftp://massive.ucsd.edu/MSV000078850/results/S....,S. cattleya,genome
3,GCA_000240165.1,RefSeq_accession,SAMN02603879,ftp://massive.ucsd.edu/MSV000078850/results/S....,S. cattleya,genome
4,GCA_000240165.1,RefSeq_accession,SAMN02603879,ftp://massive.ucsd.edu/MSV000078850/results/S....,S. cattleya,genome
...,...,...,...,...,...,...
2115,GCA_000286575.1,GenBank_accession,SAMN00255227,ftp://massive.ucsd.edu/MSV000084945/ccms_peak/...,Burkholderia multivorans CF2,genome
2116,GCA_000959505.1,GenBank_accession,SAMN03144971,ftp://massive.ucsd.edu/MSV000084945/ccms_peak/...,Burkholderia dolosa AU0158,genome
2117,GCA_000959505.1,GenBank_accession,SAMN03144971,ftp://massive.ucsd.edu/MSV000084945/ccms_peak/...,Burkholderia dolosa AU0158,genome
2118,GCA_003568605.1,GenBank_accession,SAMN02866398,ftp://massive.ucsd.edu/MSV000084945/ccms_peak/...,Burkholderia thailandensis E264,genome


In [16]:
### only run once

with open('./temp_files/genome_list_temp.txt', "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in list(filtered_df['NCBI_ID']):
        writer.writerow([val])
        
with open('./temp_files/biosample_list_temp.txt', "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in list(filtered_df['BioSample']):
        writer.writerow([val])

In [18]:
### only run once

# start = time.time()

# subprocess.call('python NCBI_getGenBankID.py --input ./temp_files/biosample_list_temp.txt --output ./outputs/biosample_list-round3-210220.out.csv', shell=True)

# end = time.time()
# hours, rem = divmod(end-start, 3600)
# minutes, seconds = divmod(rem, 60)
# print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
### 02:46:30.68

02:46:30.68


In [19]:
biosample_df = pd.read_csv('./outputs/biosample_list-round3-210220.out.csv',sep='\t')
biosample_df = biosample_df[biosample_df['GenBank_id'] != '-']
biosample_df = biosample_df.reset_index(drop=True)


biosample_df

,Accession_id,Assembly_id,GenBank_id,Refseq_id
0,SAMN02603879,376848,GCA_000240165.1,GCF_000240165.1
1,SAMN14411170,6465981,GCA_011765705.1,NaN
2,SAMN14411171,6466001,GCA_011765735.1,NaN
3,SAMD00106498,2168011,GCA_003945305.1,GCF_003945305.1
4,SAMN05710194,1533101,GCA_002897315.1,GCF_002897315.1
...,...,...,...,...
232,SAMN03144971,315111,GCA_000959505.1,GCF_000959505.1
233,SAMN00623032,39588,GCA_000018505.1,GCF_000018505.1
234,SAMN03140189,315121,GCA_000959525.1,GCF_000959525.1
235,SAMN00255227,492528,GCA_000286575.1,GCF_000286575.1


## 1.4. Adding new column with the downloaded NCBI IDs

In [20]:
len(podp_merged_df),len(filtered_df),len(np.unique(filtered_df['BioSample'])),len(np.unique(biosample_df['GenBank_id']))
### notice that we downloaded 237 out of the 259 unique BioSamples

(4896, 2120, 259, 237)

In [21]:
biosample_dict = dict(zip(biosample_df.Accession_id, biosample_df.GenBank_id))
new_col = []

for i,r in podp_merged_df.iterrows():
    biosample_id = podp_merged_df['BioSample'].loc[i]
    if biosample_id in biosample_dict.keys():
        new_col.append(biosample_dict[biosample_id])
    else:
        if 'GCA' in biosample_id or 'ERX' in biosample_id or 'ERS' in biosample_id:
            new_col.append(biosample_id)
        else:
            new_col.append('NaN')
        
podp_merged_df['New_NCBI_ID'] = new_col

podp_merged_df

,NCBI_ID,Database,BioSample,LCMS_file,Genome_label,Method,New_NCBI_ID
0,GCA_000240165.1,RefSeq_accession,SAMN02603879,ftp://massive.ucsd.edu/MSV000078850/results/S....,S. cattleya,genome,GCA_000240165.1
1,GCA_000240165.1,RefSeq_accession,SAMN02603879,ftp://massive.ucsd.edu/MSV000078850/results/S....,S. cattleya,genome,GCA_000240165.1
2,GCA_000240165.1,RefSeq_accession,SAMN02603879,ftp://massive.ucsd.edu/MSV000078850/results/S....,S. cattleya,genome,GCA_000240165.1
3,GCA_000240165.1,RefSeq_accession,SAMN02603879,ftp://massive.ucsd.edu/MSV000078850/results/S....,S. cattleya,genome,GCA_000240165.1
4,GCA_000240165.1,RefSeq_accession,SAMN02603879,ftp://massive.ucsd.edu/MSV000078850/results/S....,S. cattleya,genome,GCA_000240165.1
...,...,...,...,...,...,...,...
4891,GCA_000286575.1,GenBank_accession,SAMN00255227,ftp://massive.ucsd.edu/MSV000084945/ccms_peak/...,Burkholderia multivorans CF2,genome,GCA_000286575.1
4892,GCA_000959505.1,GenBank_accession,SAMN03144971,ftp://massive.ucsd.edu/MSV000084945/ccms_peak/...,Burkholderia dolosa AU0158,genome,GCA_000959505.1
4893,GCA_000959505.1,GenBank_accession,SAMN03144971,ftp://massive.ucsd.edu/MSV000084945/ccms_peak/...,Burkholderia dolosa AU0158,genome,GCA_000959505.1
4894,GCA_003568605.1,GenBank_accession,SAMN02866398,ftp://massive.ucsd.edu/MSV000084945/ccms_peak/...,Burkholderia thailandensis E264,genome,GCA_003568605.1


In [22]:
count = 0
gca_list = []

for item in podp_merged_df.New_NCBI_ID:
    if 'GCA' in item:
        count += 1
        gca_list.append(item)
        
print(count,len(podp_merged_df[podp_merged_df['New_NCBI_ID'] == 'NaN']),
      len(np.unique(podp_merged_df[podp_merged_df['New_NCBI_ID'] == 'NaN']['NCBI_ID'])),len(podp_merged_df))
print(len(np.unique(gca_list)))
### notice that all 237 unique BioSamples generate a GCA code and there were 490/727 other genomes that can't be downloaded

1383 2308 727 4896
237


## 1.5. Downloading and unzipping the NCBI genomes

In [23]:
# !mkdir -p /Volumes/TFL190831/NPOmix_round4/podp_ncbi/

In [24]:
cmd_list,genomes_failed = [],[]

for i,r in podp_merged_df.iterrows():
    if type(r['New_NCBI_ID']) == float:
        n = re.match(r'^\D{4}\d{2}$',r['NCBI_ID'])
        if n:
            cmd = 'wget https://sra-download.ncbi.nlm.nih.gov/traces/wgs01/wgs_aux/%s/%s/%s/%s.1.fsa_nt.gz'%(r['NCBI_ID'][0:2],r['NCBI_ID'][2:4],r['NCBI_ID'],r['NCBI_ID'])
        if 'GCA' in str(r['NCBI_ID']):
            cmd = 'wget -nd -r --no-parent -A "genomic.fna.gz"  -R "*_from_genomic*" "ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/%s/%s/%s/"'%(r['NCBI_ID'][4:7],r['NCBI_ID'][7:10],r['NCBI_ID'][10:13])
        if 'GCA' not in str(r['NCBI_ID']) and not n:
            cmd = None
    else:
        if 'GCA' in str(r['New_NCBI_ID']):
            cmd = 'wget -nd -r --no-parent -A "genomic.fna.gz"  -R "*_from_genomic*" "ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/%s/%s/%s/"'%(r['New_NCBI_ID'][4:7],r['New_NCBI_ID'][7:10],r['New_NCBI_ID'][10:13])
        else:
            cmd = None
    if cmd != None:
        if cmd not in cmd_list:
            cmd_list.append(cmd)
#             try: #only run once
#                 subprocess.check_output(cmd, shell=True, cwd='/Volumes/TFL190831/NPOmix_round4/podp_ncbi/')
#             except subprocess.CalledProcessError as e:
#                 genomes_failed.append(cmd)

print(len(cmd_list),len(genomes_failed))

237 0


In [25]:
!ls /Volumes/TFL190831/NPOmix_round4_genomes/podp_ncbi/ | wc -l

     237


In [26]:
### only run once

# genome_list = glob.glob("/Volumes/TFL190831/podp_round4_ncbi/*.gz")

# for item in genome_list:
#     genome = os.path.basename(item).split('.')[0]
#     print(os.path.basename(item))
#     cmd = "gunzip -c %s > %s.fasta"%(os.path.basename(item),genome)
#     subprocess.check_output(cmd, shell=True, cwd='/Volumes/TFL190831/NPOmix_round4_genomes/podp_ncbi/')

In [27]:
# !rm /Volumes/TFL190831/NPOmix_round4_genomes/podp_ncbi/*.gz

## 1.6. Copying previously downloaded genomes

In [28]:
glob_round3 = glob.glob('/Volumes/TFL190831/podp_round3_ncbi/*fasta')

for file_path in glob_round3:
    fasta_file = os.path.basename(file_path)
    if not os.path.exists('/Volumes/TFL190831/NPOmix_round4_genomes/podp_ncbi/%s'%fasta_file):
        print('cp %s /Volumes/TFL190831/NPOmix_round4/previous_genomes/'%file_path)
#         subprocess.call('cp %s /Volumes/TFL190831/NPOmix_round4/previous_genomes/'%file_path, shell=True)

cp /Volumes/TFL190831/podp_round3_ncbi/GCA_000025765.fasta /Volumes/TFL190831/NPOmix_round4/previous_genomes/
cp /Volumes/TFL190831/podp_round3_ncbi/GCA_000090945.fasta /Volumes/TFL190831/NPOmix_round4/previous_genomes/
cp /Volumes/TFL190831/podp_round3_ncbi/GCA_000144005.fasta /Volumes/TFL190831/NPOmix_round4/previous_genomes/
cp /Volumes/TFL190831/podp_round3_ncbi/GCA_000144025.fasta /Volumes/TFL190831/NPOmix_round4/previous_genomes/
cp /Volumes/TFL190831/podp_round3_ncbi/GCA_000144045.fasta /Volumes/TFL190831/NPOmix_round4/previous_genomes/
cp /Volumes/TFL190831/podp_round3_ncbi/GCA_000144065.fasta /Volumes/TFL190831/NPOmix_round4/previous_genomes/
cp /Volumes/TFL190831/podp_round3_ncbi/GCA_000144085.fasta /Volumes/TFL190831/NPOmix_round4/previous_genomes/
cp /Volumes/TFL190831/podp_round3_ncbi/GCA_000144105.fasta /Volumes/TFL190831/NPOmix_round4/previous_genomes/
cp /Volumes/TFL190831/podp_round3_ncbi/GCA_000144125.fasta /Volumes/TFL190831/NPOmix_round4/previous_genomes/
cp /Volume

In [29]:
!ls /Volumes/TFL190831/NPOmix_round4_genomes/podp_ncbi/*.fasta | wc -l

     237


In [30]:
!ls /Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/*.fasta | wc -l

     174


In [31]:
AS_previous_list = glob.glob('/Volumes/TFL190831/nf_output-iomega/*')
count = 0

for file_path in AS_previous_list:
    AS_file = os.path.basename(file_path).split('_output')[0]
    if os.path.exists('/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/%s'%AS_file):
        count += 1
    else:
        if os.path.exists('/Volumes/TFL190831/NPOmix_round4_genomes/podp_ncbi/%s'%AS_file):
            count += 1
            
count,len(AS_previous_list)

(279, 279)

In [32]:
previous_genomes = glob.glob('/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/*fasta')

for item in previous_genomes:
    sum_item = os.path.basename(item).split('.')[0]
    temp_df = podp_merged_df[podp_merged_df['NCBI_ID'].str.contains(sum_item)]
    if len(temp_df) == 0:
        print(item)
### for now, eventhough we don't have these genomes in the current podp_list, we already have LCMS for those

/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000025765.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000090945.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000144005.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000144025.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000144045.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000144065.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000144085.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000144105.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000144125.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000144145.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000144165.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000144185.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_00

/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000295235.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000295355.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000295415.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000295575.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000296385.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000307345.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000307375.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000307495.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000336235.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000338635.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000340725.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_000377105.fasta
/Volumes/TFL190831/NPOmix_round4_genomes/previous_genomes/GCA_00

## 1.7. Downloading metagenomic reads

In [33]:
# !mkdir /Volumes/TFL190831/NPOmix_round4/ERX_reads/
# !mkdir /Volumes/TFL190831/NPOmix_round4/ERS_reads/

In [34]:
len(np.unique(podp_merged_df[podp_merged_df['New_NCBI_ID'].str.contains('ERS')]['New_NCBI_ID']))

724

In [35]:
len(np.unique(podp_merged_df[podp_merged_df['New_NCBI_ID'].str.contains('ERX')]['New_NCBI_ID']))

481

In [36]:
### only run once
# !mkdir /Volumes/TFL190831/NPOmix_round4_genomes/ERX_fileroports/
# !mkdir /Volumes/TFL190831/NPOmix_round4_genomes/ERS_fileroports/

In [37]:
### only run once for ERS codes and one more time to ERX

# !mkdir /Volumes/TFL190831/NPOmix_round4_genomes/ERS_fileroports/

# for i,r in podp_merged_df.iterrows():
#     if 'ERS' in podp_merged_df['New_NCBI_ID'].loc[i]:
#         ena_code = podp_merged_df['New_NCBI_ID'].loc[i]
#         https_cmd = "wget \"https://www.ebi.ac.uk/ena/data/warehouse/filereport?accession=%s&result=read_run&fields=fastq_ftp\""%ena_code
#         subprocess.call(https_cmd, shell=True, cwd='/Volumes/TFL190831/NPOmix_round4_genomes/ERS_fileroports/')

In [38]:
https_list = glob.glob('/Volumes/TFL190831/NPOmix_round4_genomes/ERX_fileroports/filereport?accession=*')
cmd_list = []
ena_dict1 = {}

for item in https_list:
    https_df = pd.read_csv(item,sep='\t')
    read_1 = https_df['fastq_ftp'].loc[0].split(';')[0]
    cmd_list.append('wget ftp://%s'%read_1)
    ena_dict1[os.path.basename(read_1).split('.')[0]] = item[78:88]
    
# with open('./temp_files/ftp_ERX_list.txt', 'w') as filehandle:
#     filehandle.writelines("%s\n" % command for command in cmd_list)

In [39]:
ena_dict1

{'ERR2239508': 'ERX2291668',
 'ERR2239509': 'ERX2291669',
 'ERR2239510': 'ERX2291670',
 'ERR2239511': 'ERX2291671',
 'ERR2239512': 'ERX2291672',
 'ERR2239513': 'ERX2291673',
 'ERR2239514': 'ERX2291674',
 'ERR2239515': 'ERX2291675',
 'ERR2239428': 'ERX2291588',
 'ERR2239429': 'ERX2291589',
 'ERR2239189': 'ERX2291349',
 'ERR2239516': 'ERX2291676',
 'ERR2239517': 'ERX2291677',
 'ERR2239518': 'ERX2291678',
 'ERR2239519': 'ERX2291679',
 'ERR2239520': 'ERX2291680',
 'ERR2239521': 'ERX2291681',
 'ERR2239522': 'ERX2291682',
 'ERR2239523': 'ERX2291683',
 'ERR2239524': 'ERX2291684',
 'ERR2239525': 'ERX2291685',
 'ERR2239526': 'ERX2291686',
 'ERR2239527': 'ERX2291687',
 'ERR2239432': 'ERX2291592',
 'ERR2239528': 'ERX2291688',
 'ERR2239098': 'ERX2291258',
 'ERR2239529': 'ERX2291689',
 'ERR2239530': 'ERX2291690',
 'ERR2239433': 'ERX2291593',
 'ERR2239434': 'ERX2291594',
 'ERR2239531': 'ERX2291691',
 'ERR2239532': 'ERX2291692',
 'ERR2239435': 'ERX2291595',
 'ERR2239533': 'ERX2291693',
 'ERR2239534':

In [40]:
https_list = glob.glob('/Volumes/TFL190831/NPOmix_round4_genomes/ERS_fileroports/filereport?accession=*')
cmd_list = []
ena_dict2 = {}

for item in https_list:
    https_df = pd.read_csv(item,sep='\t')
    read_1 = https_df['fastq_ftp'].loc[0].split(';')[0]
    read_2 = https_df['fastq_ftp'].loc[0].split(';')[1]
    cmd_list.append('wget ftp://%s'%read_1)
    cmd_list.append('wget ftp://%s'%read_2)
    ena_dict2[os.path.basename(read_1).split('.')[0].split('_')[0]] = item[78:88]
    
# with open('./temp_files/ftp_ERS_list.txt', 'w') as filehandle:
#     filehandle.writelines("%s\n" % command for command in cmd_list)

In [41]:
ena_dict2

{'ERR3946693': 'ERS4346398',
 'ERR3946694': 'ERS4346399',
 'ERR3946695': 'ERS4346400',
 'ERR3946696': 'ERS4346401',
 'ERR3946697': 'ERS4346402',
 'ERR3946698': 'ERS4346403',
 'ERR3946699': 'ERS4346404',
 'ERR3946700': 'ERS4346405',
 'ERR3946701': 'ERS4346406',
 'ERR3941050': 'ERS4341364',
 'ERR3941051': 'ERS4341365',
 'ERR3946704': 'ERS4346409',
 'ERR3946705': 'ERS4346410',
 'ERR3946706': 'ERS4346411',
 'ERR3946707': 'ERS4346412',
 'ERR3946708': 'ERS4346413',
 'ERR3946709': 'ERS4346414',
 'ERR3946710': 'ERS4346415',
 'ERR3946711': 'ERS4346416',
 'ERR3946712': 'ERS4346417',
 'ERR3946713': 'ERS4346418',
 'ERR3946714': 'ERS4346419',
 'ERR3946715': 'ERS4346420',
 'ERR3941052': 'ERS4341366',
 'ERR3941053': 'ERS4341367',
 'ERR3946716': 'ERS4346421',
 'ERR3946717': 'ERS4346422',
 'ERR3946718': 'ERS4346423',
 'ERR3946719': 'ERS4346424',
 'ERR3946720': 'ERS4346425',
 'ERR3946721': 'ERS4346426',
 'ERR3946722': 'ERS4346427',
 'ERR3946723': 'ERS4346428',
 'ERR3946724': 'ERS4346429',
 'ERR3946725':

In [42]:
def Merge(dict1, dict2):
    res = {**dict1, **dict2}
    return res

ena_dict3 = Merge(ena_dict1, ena_dict2)

with open('./temp_files/ena_dict-round3-210315.csv', 'w') as f:
    for key in ena_dict3.keys():
        f.write("%s,%s\n"%(key,ena_dict3[key]))

```bash
cd /Volumes/TFL190831/NPOmix_round4_genomes/ERX_reads/

sh ./ftp_ERX_list.txt
```
and alternatively

```bash
cd /Volumes/TFL190831/NPOmix_round4_genomes/ERS_reads/

sh ./ftp_ERS_list.txt
```

In [43]:
!ls /Volumes/TFL190831/NPOmix_round4_genomes/ERS_reads/*fastq.gz | wc -l

    1448


In [44]:
!ls /Volumes/TFL190831/NPOmix_round4_genomes/ERX_reads/*fastq.gz | wc -l

     481


## 1.8. Downloading JGI (meta)genomes

`Salinispora_genomes_faa` with 119 Salinisopra was provided by the Jensen lab

In [45]:
len(np.unique(podp_merged_df[podp_merged_df['Database'].str.contains('JGI')]['NCBI_ID']))

309

In [46]:
# !mkdir /Volumes/TFL190831/NPOmix_round4_genomes/manual_genomes

In [47]:
download_col,download_list,sal_filt = [],[],[]
sal_glob = glob.glob('/Volumes/TFL190831/NPOmix_round4_genomes/Salinispora_genomes_faa/*.faa')

for item in sal_glob:
    sal_filt.append(os.path.basename(item).split('_')[0])

for i,r in podp_merged_df.iterrows():
    if 'JGI' in r['Database']:
        if 'Salinispora' in r['Genome_label']:
            sal_id = 'S' + r['Genome_label'][12] + r['Genome_label'].split(' ')[2]
            if sal_id in sal_filt:
                download_col.append('yes')
                download_list.append(sal_id)
            else:
                download_col.append('no')
        else:
            download_col.append('no')
    else:
        if r['New_NCBI_ID'] != 'NaN':
            download_col.append('yes')
        else:
            download_col.append('no')
            
podp_merged_df['Downloaded'] = download_col

In [48]:
len(np.unique(download_list)),len(download_col),len(podp_merged_df)

(112, 4896, 4896)

In [49]:
len(np.unique(podp_merged_df[podp_merged_df['Downloaded'] == 'yes']['New_NCBI_ID']))

1443

In [50]:
podp_merged_df[podp_merged_df['Downloaded'] == 'no']

,NCBI_ID,Database,BioSample,LCMS_file,Genome_label,Method,New_NCBI_ID,Downloaded
46,2821485673,JGI_Genome_ID,SAMN13061032,ftp://massive.ucsd.edu/MSV000086050/updates/20...,GUM007,metagenome-assembled genome,NaN,no
47,CP022686.1,GenBank_accession,No BioSample,ftp://massive.ucsd.edu/MSV000083387/peak/mzXML...,Ecoli_Nissle_genome,genome,NaN,no
48,CP022686.1,GenBank_accession,No BioSample,ftp://massive.ucsd.edu/MSV000083387/peak/mzXML...,Ecoli_Nissle_genome,genome,NaN,no
49,CP022686.1,GenBank_accession,No BioSample,ftp://massive.ucsd.edu/MSV000083387/peak/mzXML...,Ecoli_Nissle_genome,genome,NaN,no
50,MDRX01000000,GenBank_accession,No BioSample,ftp://massive.ucsd.edu/MSV000085021/peak/DA2-8...,WMMB235,genome,NaN,no
...,...,...,...,...,...,...,...,...
3970,"651717039, 651716642",JGI_Genome_ID,SAMN11997568,ftp://massive.ucsd.edu/MSV000083648/raw/detoxi...,Streptomyces spectabilis Dietz NRRL 2792 (ATCC...,genome,NaN,no
3971,"651717039, 651716642",JGI_Genome_ID,SAMN11997568,ftp://massive.ucsd.edu/MSV000083648/raw/detoxi...,Streptomyces spectabilis Dietz NRRL 2792 (ATCC...,genome,NaN,no
3972,"651717039, 651716642",JGI_Genome_ID,SAMN11997568,ftp://massive.ucsd.edu/MSV000083648/raw/detoxi...,Streptomyces spectabilis Dietz NRRL 2792 (ATCC...,genome,NaN,no
3973,2767802005,JGI_Genome_ID,SAMN02645517,ftp://massive.ucsd.edu/MSV000083648/raw/detoxi...,Streptomyces sp. NRRL S-325,genome,NaN,no


Use the dataframe above to manually download these genomes searching at [NCBI database](https://www.ncbi.nlm.nih.gov/Traces/wgs/?page=1&view=all&search=GCF_000196155.1) and place them a the manual_genomes folder

In [61]:
manual_glob = glob.glob('/Volumes/TFL190831/NPOmix_round4_genomes/manual_genomes/*')

manual_list,indexes_to_keep = [],[]

for item in manual_glob:
    manual_list.append(os.path.basename(item).split('.fasta')[0])

for i,r in podp_merged_df.iterrows():
    if r['Downloaded'] == 'no':
        if r['NCBI_ID'] not in manual_list:
            indexes_to_keep.append(i)

podp_merged_df.loc[indexes_to_keep].to_csv('./outputs/cant_download_podp_merged_df-round3-210315.tsv',sep='\t')

In [62]:
len(podp_merged_df[podp_merged_df['Downloaded'] == 'yes']),len(podp_merged_df[podp_merged_df['Downloaded'] == 'no']),len(podp_merged_df)

(4120, 776, 4896)

In [63]:
len(np.unique(podp_merged_df.loc[indexes_to_keep]['NCBI_ID']))

311

## 1.9. Separating Genomes without antiSMASH (Optional)

In [53]:
# !mkdir /Volumes/TFL190831/genomes_to_antiSMASH/

In [87]:
AS_all_list = glob.glob('/Volumes/TFL190831/ming_output/antismash/*')
AS_list = []
count = 0

for file_path in AS_all_list:
    AS_file = file_path.split('/')[5].split('.')[0].split('_contigs')[0]
    if AS_file in ena_dict1:
        AS_list.append(ena_dict1[AS_file])
    if AS_file in ena_dict2:
        AS_list.append(ena_dict2[AS_file])
    if AS_file not in ena_dict1 and AS_file not in ena_dict2:
        AS_list.append(AS_file)

manual_genomes = glob.glob('/Volumes/TFL190831/NPOmix_round4_genomes/manual_genomes/*')
ncbi_genomes = glob.glob('/Volumes/TFL190831/NPOmix_round4_genomes/podp_ncbi/*')
sal_genomes = glob.glob('/Volumes/TFL190831/NPOmix_round4_genomes/Salinispora_genomes_faa/*')

for item in list(manual_genomes+ncbi_genomes+sal_genomes):
    if os.path.basename(item) not in AS_list:
        count += 1
        cmd = 'cp %s /Volumes/TFL190831/genomes_to_antiSMASH/'%item
#         subprocess.call(cmd,shell=True)
        print(cmd)
    
len(AS_list),count

cp /Volumes/TFL190831/NPOmix_round4_genomes/manual_genomes/MDRX01000000.fasta /Volumes/TFL190831/genomes_to_antiSMASH/
cp /Volumes/TFL190831/NPOmix_round4_genomes/manual_genomes/2724679019.fasta /Volumes/TFL190831/genomes_to_antiSMASH/
cp /Volumes/TFL190831/NPOmix_round4_genomes/manual_genomes/2821485673.fasta /Volumes/TFL190831/genomes_to_antiSMASH/
cp /Volumes/TFL190831/NPOmix_round4_genomes/manual_genomes/2740891881.fasta /Volumes/TFL190831/genomes_to_antiSMASH/
cp /Volumes/TFL190831/NPOmix_round4_genomes/manual_genomes/2845467531.fasta /Volumes/TFL190831/genomes_to_antiSMASH/
cp /Volumes/TFL190831/NPOmix_round4_genomes/manual_genomes/2791354866.fasta /Volumes/TFL190831/genomes_to_antiSMASH/
cp /Volumes/TFL190831/NPOmix_round4_genomes/manual_genomes/2791354855.fasta /Volumes/TFL190831/genomes_to_antiSMASH/
cp /Volumes/TFL190831/NPOmix_round4_genomes/manual_genomes/NZ_BAGG00000000.1.fasta /Volumes/TFL190831/genomes_to_antiSMASH/
cp /Volumes/TFL190831/NPOmix_round4_genomes/manual_geno

(868, 676)

In [88]:
481+571

1052

In [89]:
AS_list

['ERS4341364',
 'ERS4341365',
 'ERS4341366',
 'ERS4341367',
 'ERS4341368',
 'ERS4341369',
 'ERS4341370',
 'ERS4341371',
 'ERS4341372',
 'ERS4341373',
 'ERS4341374',
 'ERS4341375',
 'ERS4341376',
 'ERS4341377',
 'ERS4341378',
 'ERS4341379',
 'ERS4341380',
 'ERS4341381',
 'ERS4341382',
 'ERS4341383',
 'ERS4341384',
 'ERS4341385',
 'ERS4341386',
 'ERS4341387',
 'ERS4341388',
 'ERS4341389',
 'ERS4341390',
 'ERS4341391',
 'ERS4341392',
 'ERS4341393',
 'ERS4341394',
 'ERS4341395',
 'ERS4341396',
 'ERS4341397',
 'ERS4341398',
 'ERS4341399',
 'ERS4341401',
 'ERS4341402',
 'ERS4341403',
 'ERS4341404',
 'ERS4341405',
 'ERS4341406',
 'ERS4341407',
 'ERS4341408',
 'ERS4341409',
 'ERS4341410',
 'ERS4341411',
 'ERS4341412',
 'ERS4341413',
 'ERS4341414',
 'ERS4341415',
 'ERS4341416',
 'ERS4341417',
 'ERS4341418',
 'ERS4341419',
 'ERS4341420',
 'ERS4341421',
 'ERS4341422',
 'ERS4341423',
 'ERS4341424',
 'ERS4341425',
 'ERS4341426',
 'ERS4341427',
 'ERS4341428',
 'ERS4341429',
 'ERS4341430',
 'ERS43414

## 1.10. Running antiSMASH

```
Ming will add code
```

## 1.11. Filtering links for (meta)genomes with antiSMASH and downloading LC-MS/MS (need update)

In [66]:
!ls /Volumes/TFL190831/ming_output/antismash/ | wc -l

     868


In [91]:
updated_col = []

for i,r in podp_merged_df.iterrows():
    if r['New_NCBI_ID'] in AS_list:
        updated_col.append('yes')
    else:
        if r['NCBI_ID'] in AS_list:
            updated_col.append('yes')
        else:
            updated_col.append('no')
        
print(len(updated_col))

4896


In [107]:
podp_merged_df['Downloaded'] = updated_col
podp_final_df = podp_merged_df[podp_merged_df['Downloaded'] == 'yes']
podp_final_df = podp_final_df.reset_index(drop=True)
# podp_final_df.to_csv('./outputs/podp_final_df-round3-210316.tsv',sep='\t')

In [96]:
podp_final_df

,NCBI_ID,Database,BioSample,LCMS_file,Genome_label,Method,New_NCBI_ID,Downloaded
0,VIKW01,GenBank_accession,SAMN12084250,ftp://massive.ucsd.edu/MSV000084950/peak/04022...,Anabaena sp. UHCC 0204,genome,GCA_009711975.1,yes
1,VILD01,GenBank_accession,SAMN12084370,ftp://massive.ucsd.edu/MSV000084950/peak/04022...,Anabaena sp. UHCC 0253,genome,GCA_009712085.1,yes
2,VILE01,GenBank_accession,SAMN12084395,ftp://massive.ucsd.edu/MSV000084950/peak/04022...,Dolichospermum planctonicum UHCC 0167,genome,GCA_009712075.1,yes
3,VILF01,GenBank_accession,SAMN12084379,ftp://massive.ucsd.edu/MSV000084950/peak/04022...,Dolichospermum flos-aquae UHCC 0037,genome,GCA_009712125.1,yes
4,VIKX01,GenBank_accession,SAMN12084251,ftp://massive.ucsd.edu/MSV000084950/peak/04022...,Dolichospermum sp. UHCC 0259,genome,GCA_009711935.1,yes
...,...,...,...,...,...,...,...,...
604,ERS4356320,ENA_NCBI_accession,ERS4356320,ftp://massive.ucsd.edu/MSV000083302/peak/mzxml...,ERS4356320,metagenome,ERS4356320,yes
605,ERS4356321,ENA_NCBI_accession,ERS4356321,ftp://massive.ucsd.edu/MSV000083302/peak/mzxml...,ERS4356321,metagenome,ERS4356321,yes
606,ERS4356322,ENA_NCBI_accession,ERS4356322,ftp://massive.ucsd.edu/MSV000083302/peak/mzxml...,ERS4356322,metagenome,ERS4356322,yes
607,ERS4356323,ENA_NCBI_accession,ERS4356323,ftp://massive.ucsd.edu/MSV000083302/peak/mzxml...,ERS4356323,metagenome,ERS4356323,yes


In [103]:
len(AS_list),len(AS_list)-len(podp_final_df) ### these 259 files are from the previous round

(868, 259)

In [104]:
podp_merged_df.shape,podp_final_df.shape

((4896, 8), (609, 8))

In [105]:
rows_wo_file,seen,commands = [],[],[]

for i,r in podp_final_df.iterrows():
    genomeID = podp_final_df['New_NCBI_ID'].loc[i]
    seen.append(genomeID)
    file_count = seen.count(genomeID)
    extension = podp_final_df['LCMS_file'].loc[i].rsplit('.', 1)[1]
    cmd = 'wget -O %s %s'%(genomeID+'.'+extension+'.'+str(file_count),podp_final_df['LCMS_file'].loc[i])
    commands.append(cmd)

commands

['wget -O GCA_009711975.1.mzML.1 ftp://massive.ucsd.edu/MSV000084950/peak/040220005.mzML',
 'wget -O GCA_009712085.1.mzML.1 ftp://massive.ucsd.edu/MSV000084950/peak/040220006.mzML',
 'wget -O GCA_009712075.1.mzML.1 ftp://massive.ucsd.edu/MSV000084950/peak/040220007.mzML',
 'wget -O GCA_009712125.1.mzML.1 ftp://massive.ucsd.edu/MSV000084950/peak/040220008.mzML',
 'wget -O GCA_009711935.1.mzML.1 ftp://massive.ucsd.edu/MSV000084950/peak/040220011.mzML',
 'wget -O GCA_009711985.1.mzML.1 ftp://massive.ucsd.edu/MSV000084950/peak/040220012.mzML',
 'wget -O GCA_009711965.1.mzML.1 ftp://massive.ucsd.edu/MSV000084950/peak/040220013.mzML',
 'wget -O GCA_009711925.1.mzML.1 ftp://massive.ucsd.edu/MSV000084950/peak/040220014.mzML',
 'wget -O GCA_009712065.1.mzML.1 ftp://massive.ucsd.edu/MSV000084950/peak/040220016.mzML',
 'wget -O GCA_009712035.1.mzML.1 ftp://massive.ucsd.edu/MSV000084950/peak/040220021.mzML',
 'wget -O GCA_009711975.1.mzML.2 ftp://massive.ucsd.edu/MSV000084950/peak/040220005.mzML',

In [106]:
### only run once

with open('./temp_files/PODP_LCMS_list_commands-round3.txt', "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in commands:
        writer.writerow([val])

```bash
mkdir /Volumes/TFL190831/podp_LCMS

cd /Volumes/TFL190831/podp_LCMS

sh PODP_LCMS_list_commands-round3.txt
```

In [108]:
downloaded_LCMS = glob.glob("/Volumes/TFL190831/podp_LCMS/*")

len(downloaded_LCMS),len(commands)

(0, 609)